In [255]:
from my_module import create_gantt_chart, parse_operations_file, parse_degradations_file
import numpy as np
import matplotlib.pyplot as plt
import random
import copy
import json

In [256]:
inf= 999999
new = 1
kacem = 'k3'
num_instance = 'instance12'
filename = f"C:/Users/gjavossevou/Desktop/Réalisations/TESTS/{kacem}/{kacem}.txt"
nbJobs, nbMachines, nbOperationsParJob, dureeOperations = parse_operations_file(filename)
filename = f"C:/Users/gjavossevou/Desktop/Réalisations/TESTS/{kacem}/{num_instance}/instance.txt"
_, _, nbComposants, seuils_degradation, dureeMaintenances, degradations = parse_degradations_file(filename)

alpha_kl = [[.005 for l in range(nbComposants[k])] for k in range(nbMachines)]
Qjmin = [0.8 for j in range(nbJobs)]
AQL = 0.1
couleurMachines = {
    'J1': '#ff9999',   # Light Red
    'J2': '#99ff99',   # Light Green
    'J3': '#9999ff',   # Light Blue
    'J4': '#61620abf', # Dark Blue
    'J5': '#ffcc99',   # Light Orange
    'J6': '#66c2a5',   # Teal
    'J7': '#fc8d62',   # Salmon
    'J8': '#8da0cb',   # Lavender
    'J9': '#e78ac3',   # Pink
    'J10': '#a6d854',  # Lime Green
    'J11': '#ffd92f',  # Yellow
    'J12': '#e5c494',  # Beige
    'J13': '#b3b3b3',  # Grey
    'J14': '#1f78b4',  # Bright Blue
    'J15': '#33a02c',  # Bright Green
    'Maintenances': '#2F4F4F'  # Dark Slate Grey
}

In [257]:
def completionTime(solution):
    maxComposants = max(nbComposants)
    iter = [0 for j in range(nbJobs)]
    i_s = [0 for j in range(nbJobs) for i in range(nbOperationsParJob[j])]
    t_ij = [[0 for i in range(nbOperationsParJob[j])] for j in range(nbJobs)]
    c_ij = [[0 for i in range(nbOperationsParJob[j])] for j in range(nbJobs)]
    D_kl = [[[0] for l in range(nbComposants[k])] for k in range(nbMachines)]
    Qj = [[1.0]  for j in range(nbJobs)]
    y = copy.deepcopy(solution[2])
    dispo_machines = [0 for _ in range(nbMachines)]
    
    for ind in range(sum(nbOperationsParJob)):
        k = solution[1][ind]
        j = solution[0][ind]
        i_s[ind] = iter[j]
        i = i_s[ind]
        if i != 0 :
            t_ij[j][i] = c_ij[j][i-1] if (c_ij[j][i-1] >= dispo_machines[k]) else dispo_machines[k]
        else :
            t_ij[j][i] = dispo_machines[k]
        c_ij[j][i] = t_ij[j][i] + dureeOperations[k][j][i]
        temp_var = 0
        for l in range(nbComposants[k]):
            ind_ = ind-1
            while(ind_ >= 0) :
                if solution[1][ind_] == k:
                    j_ = solution[0][ind_]
                    i_ = i_s[ind_]
                    if y[l][j_][i_]:
                        D_kl[k][l].append(0)
                    break
                ind_ -= 1
            temp_var += D_kl[k][l][-1]*alpha_kl[k][l]
            D_kl[k][l].append( D_kl[k][l][-1] + degradations[k][l][j][i])
            y[l][j][i] = True if(D_kl[k][l][-1] > seuils_degradation[k][l]) else y[l][j][i]
        dispo_machines[k] = c_ij[j][i] + max(y[l][j][i]*dureeMaintenances[k][l] for l in range(nbComposants[k]))
        Qj[j].append(Qj[j][-1]-temp_var)
        iter[j] += 1
    Cmax = max(c_ij[j][nbOperationsParJob[j]-1] for j in range(nbJobs))
    nbMaintenance=0
    for l in range(maxComposants):
        for j in range(nbJobs):
            for i in range(nbOperationsParJob[j]):
                if y[l][j][i]:
                    nbMaintenance += 1
    penality = 0
    quality = sum([(1-Qj[j][-1]) for j in range(nbJobs)])/nbJobs
    for j in range(nbJobs):
        if quality > AQL:
            penality += 10
    cout = 0.8*Cmax + 0.1*nbMaintenance + 0.1*penality
    return t_ij, c_ij, Cmax, D_kl, y, i_s, Qj, cout, quality

In [258]:
def create_random_solution():
    return [
        [j for j in range(nbJobs) for _ in range(nbOperationsParJob[j])],
        [random.randint(0, nbMachines-1) for _ in range(sum(nbOperationsParJob))],
        [[[False for _ in range(nbOperationsParJob[j])] for j in range(nbJobs)] for _ in range(max(nbComposants))]
    ]

In [259]:
def fitness(solution):
    _, _, Cmax, _, _, _, _, cout, _ = completionTime(solution)
    return cout

In [260]:
def crossover(selected_population):
    new_population = []
    # Mélange la population sélectionnée
    random.shuffle(selected_population)
    
    # Associer les individus par paires
    for i in range(0, len(selected_population), 2):
        if i+1 < len(selected_population):
            parent1 = selected_population[i]
            parent2 = selected_population[i+1]
            
            crossover_point1 = len(parent1[0])//2
            crossover_point2 = len(parent1[1])//2
            
            child1 = copy.deepcopy(parent1)
            child2 = copy.deepcopy(parent2)
            
            # Croisement de la première partie (ordre des jobs)
            child1[0][:crossover_point1], child2[0][:crossover_point1] = parent2[0][:crossover_point1], parent1[0][:crossover_point1]
            
            # Croisement de la deuxième partie (affectation des machines)
            child1[1][:crossover_point2], child2[1][:crossover_point2] = parent2[1][:crossover_point2], parent1[1][:crossover_point2]
            
            # Croisement de la troisième partie (plan de maintenance)
            for l in range(len(child1[2])):
                for j in range(len(child1[2][l])):
                    if random.random() < 0.5:
                        child1[2][l][j], child2[2][l][j] = child2[2][l][j], child1[2][l][j]

            new_population.extend([child1, child2])
        else:
            # Si la population sélectionnée a un nombre impair, ajouter le dernier individu sans modification
            new_population.append(selected_population[i])
    
    return new_population


In [261]:
def inserer_maintenance(y):
    # Générer des indices aléatoires
    l = random.randint(0, len(y) - 1)
    j = random.randint(0, len(y[l]) - 1)
    i = random.randint(0, len(y[l][j]) - 1)
    if random.choice([True, False]):
        y[l][j][i] = random.choice([True, False])
        
def etat_voisin(solution, nbMachines):
    voisin = copy.deepcopy(solution)
    # i, j = random.sample(range(len(voisin[0])), 2)
    # voisin[0][i], voisin[0][j] = voisin[0][j], voisin[0][i]
    k = random.randint(0, len(voisin[1]) - 1)
    voisin[1][k] = random.randint(0, nbMachines - 1)
    inserer_maintenance(voisin[2])
    return voisin
    
def mutation(population):
    new_population = []
    for sol in population:
        new_sol = copy.deepcopy(sol)
        if random.random() < 0.1:
            new_sol = etat_voisin(sol, nbMachines)
        new_population.append(new_sol)
    return new_population

In [262]:
pop_size = 100
generations = 1000
mutation_rate = 0.1
population = [create_random_solution() for _ in range(pop_size)]

In [263]:
for generation in range(generations):
    # Évaluer la fitness
    fitness_scores = [fitness(sol) for sol in population]
    # Sélectionner les meilleurs
    selected_population = [population[i] for i in sorted(range(pop_size), key=lambda x: fitness_scores[x])[:pop_size//2]]
    
    # Croisement
    new_population = crossover(selected_population)
    
    # Mutation
    new_population = mutation(new_population)
    
    # Remplacement
    population = selected_population + new_population

In [264]:
opt_solution = min(population, key=fitness)

In [265]:
def creerDiagrammeGantt(solution, dureeOperations, couleurMachines):
    taches = []
    ########## Provisoire à changer apres optimisation
    t_ij, c_ij, Cmax, deg, y, i_s, Qj, nbMaintenance, quality = completionTime(solution)
    ###########
    for ind in range(sum(nbOperationsParJob)):
        k = solution[1][ind]
        j = solution[0][ind]
        i = i_s[ind]
        start = t_ij[j][i]
        end = c_ij[j][i]
        machine = k
        tache = dict(task=f"Machine {machine+1}",
                     start=start,
                     end=end,
                     rsc=f"J{j+1}",
                     label=f"O{j+1}:{i+1}",
                     info=f"J{j+1}")
        taches.append(tache)
    ############
    for ind in range(sum(nbOperationsParJob)):
        for k in range(nbMachines):
            for l in range(nbComposants[k]):
                i = i_s[ind]
                j = solution[0][ind]
                k_ = solution[1][ind]
                if (y[l][j][i] and k == k_):
                    for ind_ in range(ind+1, sum(nbOperationsParJob)):
                        if solution[1][ind_] == k_ :
                            machine = k+1
                            composant = l+1
                            start_time = c_ij[j][i]
                            finish_time = start_time + dureeMaintenances[k][l]
                            tache = dict(task=f"Machine {machine}", 
                                         start=start_time, 
                                         end=finish_time, 
                                         rsc=f"Maintenances",
                                         label=f"M", 
                                         info=f"Composant : {composant} (durée={dureeMaintenances[k][l]})")
                            taches.append(tache)
                            break
            
    taches = sorted(taches, key=lambda x: x['task'])
    print("Valeur du Cmax : ", Cmax)
    return taches, deg, y, i_s, Qj, Cmax, quality

In [267]:
fig, deg, y, i_s, Qj, Cmax, quality = creerDiagrammeGantt(opt_solution, dureeOperations, couleurMachines)
app = create_gantt_chart(couleurMachines, fig, title="Ordonnancement", cell_widht=30, cell_height=40)


for elt in deg :
    for elt_ in elt:
        print(elt_)
print('########################################')
print('########################################')
for elt in y:
    for elt_ in elt:
        print(elt_)
print('########################################')
for elt in Qj:
    print(elt)
    
data = {
    "Cmax_x": Cmax,
    "fig": fig,
    "quality": Qj,
    "degradations": deg
}

if __name__ == '__main__':
    app.run_server(debug=True, port=8080)

Valeur du Cmax :  24
[0, 8, 14, 0, 4]
[0, 3, 11, 15]
[0, 1, 4, 10, 12]
[0, 4, 5, 10, 12]
[0, 5, 7]
[0, 10, 15]
[0, 8, 13]
[0, 2, 6, 8]
[0, 5, 8, 16]
[0, 10, 12, 13]
[0, 4, 7]
[0, 9, 10]
[0, 8, 13]
[0, 5, 8]
[0, 4, 9, 15, 0, 8]
[0, 2, 8, 13, 17, 0, 8, 10]
[0, 2, 6, 10, 18, 0, 2, 7]
[0, 6, 13]
[0, 7, 14]
[0, 7, 8]
[0, 4, 14]
[0, 5, 6]
[0, 9, 10]
[0, 1, 9]
[0, 3, 9]
[0, 9, 14]
########################################
########################################
[False, False, False]
[False, False, True]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[True, False, True]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, True]
[False, False, True]
[False, False, False]
[False, False, True]
[False, False, False]
[False, True, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False]
[False, False, False

In [269]:
for mac in seuils_degradation:
    for comp in mac :
        print(comp)

13
13
12
16
12
11
14
16
16
13
11
16
15
16
10
13
12
14
16
17
11
14
17
14
10
14
